In [10]:
#SCRAPING ENVOIREMENT

In [2]:
#Importing libaries
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
import re
import itertools
import random
import threading
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import csv

## De følgende celler skal IKKE køres

In [447]:
#Importing page-links
"""
list_url = []
for page in range(1,201,1):
    url = f'https://www.boligsiden.dk/tilsalg/villa,ejerlejlighed,raekkehus,landejendom?page={page}'
    list_url.append(url)

print(len(list_url))
"""




200


In [449]:
#Retriving HTMLS
"""
list_htmls =[]
for url in list_url:
    response = requests.get(url, headers={'name': 'Mathias Rasmussen', 'email': 'kbd655@alumni.ku.dk'})
    list_htmls.append(response.text)
    time.sleep(0.5)

"""
    

In [451]:
# Finding links on appartments
"""
links = []
for html in list_htmls:
    soup = BeautifulSoup(html, 'lxml')
    pattern = re.compile(r'href\":\"(/addresses/[^\"]+)')
    matches = pattern.findall(str(soup))

    apartments_list = list(set(matches))


    base_url = "https://www.boligsiden.dk"
    apartment_links = [base_url + link for link in apartments_list] #This list contains appartments
    links.append(apartment_links)

flattened_links = list(itertools.chain(*links))
print(len(flattened_links)) #Counting number of links
"""

9996


In [453]:
#Saving list of links to be scrapped
"""
with open('rawdata/links.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    for link in flattened_links:
        writer.writerow([link])
"""


# HER STARTER SCRAPING PROCESSEN

In [4]:
#Importing relevant list of links:
flattened_links = []

with open('rawdata/links_version_2.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        flattened_links.append(row[0])  # Assuming each link is in the first column

print(len(flattened_links))

9999


In [6]:
op = webdriver.ChromeOptions()
op.add_argument('headless')
driver = webdriver.Chrome(options=op)

In [8]:
#Carefull! Runs with X-links at a time
#NANNAS VERSION
dict = {}
links_med_fejl = []

def fetch_data(link):
    driver = webdriver.Chrome(options=op)
    try:
        driver.get(link)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        time.sleep(random.uniform(2, 3))  # Reducing sleep time slightly
        area_size = soup.find_all('span', class_='text-gray-800 whitespace-nowrap cursor-pointer border-b border-gray-400 pb-0.5 border-dotted text-sm')
        other = soup.find_all('span', class_="text-gray-800 whitespace-nowrap border-dotted text-sm")
        address = soup.find_all('div', class_='font-bold text-sm md:text-base')
        pstnr = soup.find_all('div', class_="mt-1 text-xs md:text-sm text-gray-600")
        pstnr_ny = re.findall(r'\d+', pstnr[0].get_text())  # Searching for numbers in postal-code string.
        try:
            total_address = address[0].get_text() + ',' + pstnr_ny[0]
        except:
            print('adresseproblem')
        #Nannas kode:
        try:    
            price = soup.find('h2', class_='text-2xl xl:text-3xl text-blue-900 font-bold').get_text()
        except:
            price = "fejl"
            
        #Finder label hvor afstand er under
        """
        try:
            school = soup.find('div', string='Afstand').find_next_sibling('div').get_text()
        except:
            school = "fejl"
        #Karakter
        try:
            grades = soup.find('div', string='Karaktergennemsnit').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            grades = "fejl"
        #Trivsel
        try:
            trivsel = soup.find('div', string='Trivsel').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            trivsel = "fejl"
        """
        try:
            boligtype = soup.find('span', class_='text-black text-sm pr-2').get_text()
        except:
            boligtype = "fejl"
        #energi = soup.find('svg', id='Lag_1').find('title')
        try:
            energi = soup.find('title', string=re.compile('Energimærke')).get_text()
        except:
            energi = "fejl"
        try:
            radon = soup.find('div', class_='h-7 text-sm font-bold').get_text()
        except:
            radon = "fejl"
    
        #kommune_info og liggetid med text behandling
        try:
            kommune_info = soup.find('div', class_='sec_municipality_info')
            if kommune_info:
                kommune_text = kommune_info.get_text()
        
                if 'Kommuneskatten er på ' in kommune_text:
                    kommuneskat = kommune_text.split('Kommuneskatten er på ')[1].split(' ')[0]
                else:
                    kommuneskat = "fejl"
        
                if 'grundskyldspromillen på ' in kommune_text:
                    grundskyldspromillen = kommune_text.split('grundskyldspromillen på ')[1].split(' ')[0]
                else:
                    grundskyldspromillen = "fejl"
        except:
            kommuneskat = "fejl"
            grundskyldspromillen = "fejl"
            
            
        
        try:
            liggetid = soup.find('div', class_='pb-0.5 border-dashed border-b border-gray-300 cursor-pointer text-sm text-gray-800')
            if liggetid:
                liggetid_text = liggetid.get_text(separator=" ", strip=True).replace(' :', ':')
        except:
            liggetid_text = "fejl"

        #################   NY KODE med skoler    ########################
        #Skolenavn
        #Skolenavn
        try:
            school_name_1 = soup.find('div', class_='text-sm font-bold text-blue-900').get_text()
        except:
            school_name_1 = "fejl"
        try:
            school_name_2 = soup.find('div', class_='text-sm font-bold text-blue-900').find_next('div', class_='text-sm font-bold text-blue-900').get_text()
        except:
            school_name_2 = "fejl"
        try:
            school_name_3 = soup.find('div', class_='text-sm font-bold text-blue-900').find_next('div', class_='text-sm font-bold text-blue-900').find_next('div', class_='text-sm font-bold text-blue-900').get_text()
        except:
            school_name_3 = "fejl"

        #Afstand 
        try:
            school_1 = soup.find('div', string='Afstand').find_next_sibling('div').get_text()
        except:
            school_1 = "fejl"
        try:
            school_2 = soup.find('div', string='Afstand').find_next('div', string='Afstand').find_next_sibling('div').get_text()
        except:
            school_2 = "fejl"
        try:
            school_3 = soup.find('div', string='Afstand').find_next('div', string='Afstand').find_next('div', string='Afstand').find_next_sibling('div').get_text()
        except:
            school_3 = "fejl"

        #Karakterer
        try:
            grades_1 = soup.find('div', string='Karaktergennemsnit').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            grades_1 = "fejl"
        try:
            grades_2 = soup.find('div', string='Karaktergennemsnit').find_next('div', string='Karaktergennemsnit').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            grades_2 = "fejl"
        try:
            grades_3 = soup.find('div', string='Karaktergennemsnit').find_next('div', string='Karaktergennemsnit').find_next('div', string='Karaktergennemsnit').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            grades_3 = "fejl"

        #Trivsel
        try:
            trivsel_1 = soup.find('div', string='Trivsel').find_next_sibling('div').find_next_sibling('div').get_text()
        except: 
            trivsel_1 = "fejl"
        try:
            trivsel_2 = soup.find('div', string='Trivsel').find_next('div', string='Trivsel').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            trivsel_2 = "fejl"
        try:
            trivsel_3 = soup.find('div', string='Trivsel').find_next('div', string='Trivsel').find_next('div', string='Trivsel').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            trivsel_3 = "fejl"

        #ENDELIGE 
        
        if grades_1 not in ["fejl", "Ikke oplyst"]:
            school_name=school_name_1
            school = school_1
            grades = grades_1
            trivsel = trivsel_1
        elif grades_2 not in ["fejl", "Ikke oplyst"]:
            school_name=school_name_2
            school = school_2
            grades = grades_2
            trivsel = trivsel_2
        elif grades_3 not in ["fejl", "Ikke oplyst"]:
            school_name=school_name_3
            school = school_3
            grades = grades_3
            trivsel = trivsel_3
        else:
            school_name=school_name_1
            school = school_1 # Fallback to school_1 if all are "fejl" or "Ikke oplyst"
            grades = grades_1
            trivsel = trivsel_1



        
        try:
            if len(area_size) > 2:  # Bolig
                dict[total_address] = {'Price': price, 'Schoolname': school_name, 'Distance to school': school, 'School-grades': grades, 'Trivsel': trivsel, 'Boligtype': boligtype, 'Energy-classification': energi, 'Radon': radon, 'Kommuneskat': kommuneskat, 'Grundskyldspromillen': grundskyldspromillen, 'Kommune tekst': kommune_text, 'Liggetid': liggetid_text, 'm2': area_size[0].get_text(), 'Grund': area_size[1].get_text(), 'Ejerudgift': area_size[2].get_text(),
                                       'Rooms': other[0].get_text(), 'Build': other[1].get_text(), 'Toilets': other[2].get_text(), 'Postal code': pstnr_ny[0], 'link': link}
            else:  # Lejlighed
                dict[total_address] = {'Price': price, 'Schoolname': school_name, 'Distance to school': school, 'School-grades': grades, 'Trivsel': trivsel, 'Boligtype': boligtype, 'Energy-classification': energi, 'Radon': radon, 'Kommuneskat': kommuneskat, 'Grundskyldspromillen': grundskyldspromillen, 'Kommune tekst': kommune_text, 'Liggetid': liggetid_text, 'm2': area_size[0].get_text(), 'Ejerudgift': area_size[1].get_text(), 'Elevator': other[0].get_text(),
                                       'Rooms': other[1].get_text(), 'Build': other[2].get_text(), 'Toilets': other[3].get_text(),
                                       'Altan': other[4].get_text(), 'Terasse': other[5].get_text(), 'Postal code': pstnr_ny[0], 'link': link}
        except:
            dict[total_address] = {'m2': 'fejl', 'Ejerudgift': 'fejl', 'Elevator': 'fejl', 'Rooms': 'fejl', 'Build': 'fejl', 'Toilet': 'fejl', 'Altan': 'fejl', 'Terasse': 'fejl', 'Postal code': 'fejl', 'link': link}
    except Exception as e:
        print(f'Error occurred for link: {link} - {e}')
        links_med_fejl.append(link)

# Using ThreadPoolExecutor to handle multiple requests concurrently
with ThreadPoolExecutor(max_workers=5) as executor:
    list(tqdm(executor.map(fetch_data, flattened_links[0:1000]), total=len(flattened_links[0:1000])))


  2%|▉                                        | 23/1000 [00:32<17:59,  1.10s/it]

Error occurred for link: https://www.boligsiden.dk/addresses/0a3f50ba-4a93-32b8-e044-0003ba298018 - list index out of range


  6%|██▌                                      | 64/1000 [01:20<17:48,  1.14s/it]

Error occurred for link: https://www.boligsiden.dk/addresses/51d741bf-91fb-4e86-870f-4f4a832f31f7 - list index out of range


 13%|█████▎                                  | 134/1000 [02:44<17:09,  1.19s/it]

...

 96%|██████████████████████████████████████▍ | 961/1000 [22:03<00:44,  1.14s/it]

Error occurred for link: https://www.boligsiden.dk/addresses/0a3f50b7-e90d-32b8-e044-0003ba298018 - list index out of range


 97%|██████████████████████████████████████▉ | 972/1000 [22:17<00:33,  1.20s/it]

Error occurred for link: https://www.boligsiden.dk/addresses/3cb0a0c6-39dc-0e4a-e044-0003ba298018 - list index out of range
Error occurred for link: https://www.boligsiden.dk/addresses/0a3f50b0-4371-32b8-e044-0003ba298018 - list index out of range


100%|███████████████████████████████████████| 1000/1000 [23:33<00:00,  1.41s/it]


In [2]:
#Sikkerhedskopi
"""
#Carefull! Runs with X-links at a time
dict = {}
links_med_fejl = []

def fetch_data(link):
    #service = Service(ChromeDriverManager().install())
    #driver = webdriver.Chrome(service=service)
    driver = webdriver.Chrome(options=op)
    try:
        driver.get(link)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        time.sleep(random.uniform(2, 3))  # Reducing sleep time slightly
        area_size = soup.find_all('span', class_='text-gray-800 whitespace-nowrap cursor-pointer border-b border-gray-400 pb-0.5 border-dotted text-sm')
        other = soup.find_all('span', class_="text-gray-800 whitespace-nowrap border-dotted text-sm")
        address = soup.find_all('div', class_='font-bold text-sm md:text-base')
        pstnr = soup.find_all('div', class_="mt-1 text-xs md:text-sm text-gray-600")
        pstnr_ny = re.findall(r'\d+', pstnr[0].get_text())  # Searching for numbers in postal-code string.
        try:
            total_address = address[0].get_text() + ',' + pstnr_ny[0]
        except:
            print('adresseproblem')
        
        #Nannas kode:
        try:    
            price = soup.find('h2', class_='text-2xl xl:text-3xl text-blue-900 font-bold').get_text()
        except:
            price = "fejl"
        #Finder label hvor afstand er under
        try:
            school = soup.find('div', string='Afstand').find_next_sibling('div').get_text()
        except:
            school = "fejl"
        #Karakter
        try:
            grades = soup.find('div', string='Karaktergennemsnit').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            grades = "fejl"
        #Trivsel
        try:
            trivsel = soup.find('div', string='Trivsel').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            trivsel = "fejl"
        try:
            boligtype = soup.find('span', class_='text-black text-sm pr-2').get_text()
        except:
            boligtype = "fejl"
        #energi = soup.find('svg', id='Lag_1').find('title')
        try:
            energi = soup.find('title', string=re.compile('Energimærke')).get_text()
        except:
            energi = "fejl"
        try:
            radon = soup.find('div', class_='h-7 text-sm font-bold').get_text()
        except:
            radon = "fejl"
    
        #kommune_info og liggetid med text behandling
        try:
            kommune_info = soup.find('div', class_='sec_municipality_info')
            if kommune_info:
                kommune_text = kommune_info.get_text()
        
                if 'Kommuneskatten er på ' in kommune_text:
                    kommuneskat = kommune_text.split('Kommuneskatten er på ')[1].split(' ')[0]
                else:
                    kommuneskat = "fejl"
        
                if 'grundskyldspromillen på ' in kommune_text:
                    grundskyldspromillen = kommune_text.split('grundskyldspromillen på ')[1].split(' ')[0]
                else:
                    grundskyldspromillen = "fejl"
        except:
            kommuneskat = "fejl"
            grundskyldspromillen = "fejl"
            
            
        
        try:
            liggetid = soup.find('div', class_='pb-0.5 border-dashed border-b border-gray-300 cursor-pointer text-sm text-gray-800')
            if liggetid:
                liggetid_text = liggetid.get_text(separator=" ", strip=True).replace(' :', ':')
        except:
            liggetid_text = "fejl"


        
        try:
            if len(area_size) > 2:  # Bolig
                dict[total_address] = {'Price': price, 'Distance to school': school, 'School-grades': grades, 'Trivsel': trivsel, 'Boligtype': boligtype, 'Energy-classification': energi, 'Radon': radon, 'Kommuneskat': kommuneskat, 'Grundskyldspromillen': grundskyldspromillen, 'Liggetid': liggetid_text, 'Kommune_tekst': kommune_text, 'm2': area_size[0].get_text(), 'Grund': area_size[1].get_text(), 'Ejerudgift': area_size[2].get_text(),
                                       'Rooms': other[0].get_text(), 'Build': other[1].get_text(), 'Toilets': other[2].get_text(), 'Postal code': pstnr_ny[0], 'link': link}
            else:  # Lejlighed
                dict[total_address] = {'Price': price, 'Distance to school': school, 'School-grades': grades, 'Trivsel': trivsel, 'Boligtype': boligtype, 'Energy-classification': energi, 'Radon': radon, 'Kommuneskat': kommuneskat, 'Grundskyldspromillen': grundskyldspromillen, 'Liggetid': liggetid_text, 'Kommune_tekst': kommune_text, 'm2': area_size[0].get_text(), 'Ejerudgift': area_size[1].get_text(), 'Elevator': other[0].get_text(),
                                       'Rooms': other[1].get_text(), 'Build': other[2].get_text(), 'Toilets': other[3].get_text(),
                                       'Altan': other[4].get_text(), 'Terasse': other[5].get_text(), 'Postal code': pstnr_ny[0], 'link': link}
        except:
            dict[total_address] = {'m2': 'fejl', 'Ejerudgift': 'fejl', 'Elevator': 'fejl', 'Rooms': 'fejl', 'Build': 'fejl', 'Toilet': 'fejl', 'Altan': 'fejl', 'Terasse': 'fejl', 'Postal code': 'fejl', 'link': link}
    except Exception as e:
        print(f'Error occurred for link: {link} - {e}')
        links_med_fejl.append(link)

# Using ThreadPoolExecutor to handle multiple requests concurrently
with ThreadPoolExecutor(max_workers=5) as executor:
    list(tqdm(executor.map(fetch_data, flattened_links[0:50]), total=len(flattened_links)))
"""

NameError: name 'ThreadPoolExecutor' is not defined

In [10]:
#Denne celle henter data på alle links med fejl fra ovenstående celle! Kør derfor efter ovenstående.
def fetch_data(link):
    driver = webdriver.Chrome(options=op)
    try:
        driver.get(link)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        time.sleep(random.uniform(2, 3))  # Reducing sleep time slightly
        area_size = soup.find_all('span', class_='text-gray-800 whitespace-nowrap cursor-pointer border-b border-gray-400 pb-0.5 border-dotted text-sm')
        other = soup.find_all('span', class_="text-gray-800 whitespace-nowrap border-dotted text-sm")
        address = soup.find_all('div', class_='font-bold text-sm md:text-base')
        pstnr = soup.find_all('div', class_="mt-1 text-xs md:text-sm text-gray-600")
        pstnr_ny = re.findall(r'\d+', pstnr[0].get_text())  # Searching for numbers in postal-code string.
        try:
            total_address = address[0].get_text() + ',' + pstnr_ny[0]
        except:
            print('adresseproblem')
        #Nannas kode:
        try:    
            price = soup.find('h2', class_='text-2xl xl:text-3xl text-blue-900 font-bold').get_text()
        except:
            price = "fejl"
    
        try:
            boligtype = soup.find('span', class_='text-black text-sm pr-2').get_text()
        except:
            boligtype = "fejl"
        #energi = soup.find('svg', id='Lag_1').find('title')
        try:
            energi = soup.find('title', string=re.compile('Energimærke')).get_text()
        except:
            energi = "fejl"
        try:
            radon = soup.find('div', class_='h-7 text-sm font-bold').get_text()
        except:
            radon = "fejl"
    
        #kommune_info og liggetid med text behandling
        try:
            kommune_info = soup.find('div', class_='sec_municipality_info')
            if kommune_info:
                kommune_text = kommune_info.get_text()
        
                if 'Kommuneskatten er på ' in kommune_text:
                    kommuneskat = kommune_text.split('Kommuneskatten er på ')[1].split(' ')[0]
                else:
                    kommuneskat = "fejl"
        
                if 'grundskyldspromillen på ' in kommune_text:
                    grundskyldspromillen = kommune_text.split('grundskyldspromillen på ')[1].split(' ')[0]
                else:
                    grundskyldspromillen = "fejl"
        except:
            kommuneskat = "fejl"
            grundskyldspromillen = "fejl"
            
            
        
        try:
            liggetid = soup.find('div', class_='pb-0.5 border-dashed border-b border-gray-300 cursor-pointer text-sm text-gray-800')
            if liggetid:
                liggetid_text = liggetid.get_text(separator=" ", strip=True).replace(' :', ':')
        except:
            liggetid_text = "fejl"

        
        #################   NY KODE med skoler    ########################
        #Skolenavn
        try:
            school_name_1 = soup.find('div', class_='text-sm font-bold text-blue-900').get_text()
        except:
            school_name_1 = "fejl"
        try:
            school_name_2 = soup.find('div', class_='text-sm font-bold text-blue-900').find_next('div', class_='text-sm font-bold text-blue-900').get_text()
        except:
            school_name_2 = "fejl"
        try:
            school_name_3 = soup.find('div', class_='text-sm font-bold text-blue-900').find_next('div', class_='text-sm font-bold text-blue-900').find_next('div', class_='text-sm font-bold text-blue-900').get_text()
        except:
            school_name_3 = "fejl"

        #Afstand 
        try:
            school_1 = soup.find('div', string='Afstand').find_next_sibling('div').get_text()
        except:
            school_1 = "fejl"
        try:
            school_2 = soup.find('div', string='Afstand').find_next('div', string='Afstand').find_next_sibling('div').get_text()
        except:
            school_2 = "fejl"
        try:
            school_3 = soup.find('div', string='Afstand').find_next('div', string='Afstand').find_next('div', string='Afstand').find_next_sibling('div').get_text()
        except:
            school_3 = "fejl"

        #Karakterer
        try:
            grades_1 = soup.find('div', string='Karaktergennemsnit').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            grades_1 = "fejl"
        try:
            grades_2 = soup.find('div', string='Karaktergennemsnit').find_next('div', string='Karaktergennemsnit').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            grades_2 = "fejl"
        try:
            grades_3 = soup.find('div', string='Karaktergennemsnit').find_next('div', string='Karaktergennemsnit').find_next('div', string='Karaktergennemsnit').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            grades_3 = "fejl"

        #Trivsel
        try:
            trivsel_1 = soup.find('div', string='Trivsel').find_next_sibling('div').find_next_sibling('div').get_text()
        except: 
            trivsel_1 = "fejl"
        try:
            trivsel_2 = soup.find('div', string='Trivsel').find_next('div', string='Trivsel').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            trivsel_2 = "fejl"
        try:
            trivsel_3 = soup.find('div', string='Trivsel').find_next('div', string='Trivsel').find_next('div', string='Trivsel').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            trivsel_3 = "fejl"

        #ENDELIGE 
        
        if grades_1 not in ["fejl", "Ikke oplyst"]:
            school_name=school_name_1
            school = school_1
            grades = grades_1
            trivsel = trivsel_1
        elif grades_2 not in ["fejl", "Ikke oplyst"]:
            school_name=school_name_2
            school = school_2
            grades = grades_2
            trivsel = trivsel_2
        elif grades_3 not in ["fejl", "Ikke oplyst"]:
            school_name=school_name_3
            school = school_3
            grades = grades_3
            trivsel = trivsel_3
        else:
            school_name=school_name_1
            school = school_1 # Fallback to school_1 if all are "fejl" or "Ikke oplyst"
            grades = grades_1
            trivsel = trivsel_1


        
        try:
            if len(area_size) > 2:  # Bolig
                dict[total_address] = {'Price': price, 'Schoolname': school_name, 'Distance to school': school, 'School-grades': grades, 'Trivsel': trivsel, 'Boligtype': boligtype, 'Energy-classification': energi, 'Radon': radon, 'Kommuneskat': kommuneskat, 'Grundskyldspromillen': grundskyldspromillen, 'Kommune tekst': kommune_text, 'Liggetid': liggetid_text, 'm2': area_size[0].get_text(), 'Grund': area_size[1].get_text(), 'Ejerudgift': area_size[2].get_text(),
                                       'Rooms': other[0].get_text(), 'Build': other[1].get_text(), 'Toilets': other[2].get_text(), 'Postal code': pstnr_ny[0], 'link': link}
            else:  # Lejlighed
                dict[total_address] = {'Price': price, 'Schoolname': school_name, 'Distance to school': school, 'School-grades': grades, 'Trivsel': trivsel, 'Boligtype': boligtype, 'Energy-classification': energi, 'Radon': radon, 'Kommuneskat': kommuneskat, 'Grundskyldspromillen': grundskyldspromillen, 'Kommune tekst': kommune_text, 'Liggetid': liggetid_text, 'm2': area_size[0].get_text(), 'Ejerudgift': area_size[1].get_text(), 'Elevator': other[0].get_text(),
                                       'Rooms': other[1].get_text(), 'Build': other[2].get_text(), 'Toilets': other[3].get_text(),
                                       'Altan': other[4].get_text(), 'Terasse': other[5].get_text(), 'Postal code': pstnr_ny[0], 'link': link}
        except:
            dict[total_address] = {'m2': 'fejl', 'Ejerudgift': 'fejl', 'Elevator': 'fejl', 'Rooms': 'fejl', 'Build': 'fejl', 'Toilet': 'fejl', 'Altan': 'fejl', 'Terasse': 'fejl', 'Postal code': 'fejl', 'link': link}
    except Exception as e:
        print(f'Error occurred for link: {link} - {e}')
        links_med_fejl.append(link)

# Using ThreadPoolExecutor to handle multiple requests concurrently
with ThreadPoolExecutor(max_workers=1) as executor:
    list(tqdm(executor.map(fetch_data, links_med_fejl), total=len(links_med_fejl)))


  6%|██▊                                         | 1/16 [00:05<01:28,  5.88s/it]

Error occurred for link: https://www.boligsiden.dk/addresses/0a3f50ba-4a93-32b8-e044-0003ba298018 - list index out of range


100%|███████████████████████████████████████████| 16/16 [01:24<00:00,  5.27s/it]


In [376]:
#sikkerheds

"""
#Denne celle henter data på alle links med fejl fra ovenstående celle! Kør derfor efter ovenstående.
def fetch_data(link):
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    try:
        driver.get(link)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        time.sleep(random.uniform(2, 3))  # Reducing sleep time slightly
        area_size = soup.find_all('span', class_='text-gray-800 whitespace-nowrap cursor-pointer border-b border-gray-400 pb-0.5 border-dotted text-sm')
        other = soup.find_all('span', class_="text-gray-800 whitespace-nowrap border-dotted text-sm")
        address = soup.find_all('div', class_='font-bold text-sm md:text-base')
        pstnr = soup.find_all('div', class_="mt-1 text-xs md:text-sm text-gray-600")
        pstnr_ny = re.findall(r'\d+', pstnr[0].get_text())  # Searching for numbers in postal-code string.
        try:
            total_address = address[0].get_text() + ',' + pstnr_ny[0]
        except:
            print('adresseproblem')
        #Nannas kode:
        try:    
            price = soup.find('h2', class_='text-2xl xl:text-3xl text-blue-900 font-bold').get_text()
        except:
            price = "fejl"
        #Finder label hvor afstand er under
        try:
            school = soup.find('div', string='Afstand').find_next_sibling('div').get_text()
        except:
            school = "fejl"
        #Karakter
        try:
            grades = soup.find('div', string='Karaktergennemsnit').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            grades = "fejl"
        #Trivsel
        try:
            trivsel = soup.find('div', string='Trivsel').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            trivsel = "fejl"
        try:
            boligtype = soup.find('span', class_='text-black text-sm pr-2').get_text()
        except:
            boligtype = "fejl"
        #energi = soup.find('svg', id='Lag_1').find('title')
        try:
            energi = soup.find('title', string=re.compile('Energimærke')).get_text()
        except:
            energi = "fejl"
        try:
            radon = soup.find('div', class_='h-7 text-sm font-bold').get_text()
        except:
            radon = "fejl"
    
        #kommune_info og liggetid med text behandling
        try:
            kommune_info = soup.find('div', class_='sec_municipality_info')
            if kommune_info:
                kommune_text = kommune_info.get_text()
        
                if 'Kommuneskatten er på ' in kommune_text:
                    kommuneskat = kommune_text.split('Kommuneskatten er på ')[1].split(' ')[0]
                else:
                    kommuneskat = "fejl"
        
                if 'grundskyldspromillen på ' in kommune_text:
                    grundskyldspromillen = kommune_text.split('grundskyldspromillen på ')[1].split(' ')[0]
                else:
                    grundskyldspromillen = "fejl"
        except:
            kommuneskat = "fejl"
            grundskyldspromillen = "fejl"
            
            
        
        try:
            liggetid = soup.find('div', class_='pb-0.5 border-dashed border-b border-gray-300 cursor-pointer text-sm text-gray-800')
            if liggetid:
                liggetid_text = liggetid.get_text(separator=" ", strip=True).replace(' :', ':')
        except:
            liggetid_text = "fejl"


        
        try:
            if len(area_size) > 2:  # Bolig
                dict[total_address] = {'Price': price, 'Distance to school': school, 'School-grades': grades, 'Trivsel': trivsel, 'Boligtype': boligtype, 'Energy-classification': energi, 'Radon': radon, 'Kommuneskat': kommuneskat, 'Grundskyldspromillen': grundskyldspromillen, 'Liggetid': liggetid_text, 'm2': area_size[0].get_text(), 'Grund': area_size[1].get_text(), 'Ejerudgift': area_size[2].get_text(),
                                       'Rooms': other[0].get_text(), 'Build': other[1].get_text(), 'Toilets': other[2].get_text(), 'Postal code': pstnr_ny[0], 'link': link}
            else:  # Lejlighed
                dict[total_address] = {'Price': price, 'Distance to school': school, 'School-grades': grades, 'Trivsel': trivsel, 'Boligtype': boligtype, 'Energy-classification': energi, 'Radon': radon, 'Kommuneskat': kommuneskat, 'Grundskyldspromillen': grundskyldspromillen, 'Liggetid': liggetid_text, 'm2': area_size[0].get_text(), 'Ejerudgift': area_size[1].get_text(), 'Elevator': other[0].get_text(),
                                       'Rooms': other[1].get_text(), 'Build': other[2].get_text(), 'Toilets': other[3].get_text(),
                                       'Altan': other[4].get_text(), 'Terasse': other[5].get_text(), 'Postal code': pstnr_ny[0], 'link': link}
        except:
            dict[total_address] = {'m2': 'fejl', 'Ejerudgift': 'fejl', 'Elevator': 'fejl', 'Rooms': 'fejl', 'Build': 'fejl', 'Toilet': 'fejl', 'Altan': 'fejl', 'Terasse': 'fejl', 'Postal code': 'fejl', 'link': link}
    except Exception as e:
        print(f'Error occurred for link: {link} - {e}')
        links_med_fejl.append(link)

# Using ThreadPoolExecutor to handle multiple requests concurrently
with ThreadPoolExecutor(max_workers=1) as executor:
    list(tqdm(executor.map(fetch_data, links_med_fejl), total=len(links_med_fejl)))
""""

NameError: name 'liste_med_fejl' is not defined

## Lav først til DataFrame og efterfølgende til CSV fil

In [16]:
#Converting dict to dataframe:
DF_NAME = 'index_10000_11000' #Her kalder du din Dataframe/CSV-fil noget unikt.

df_DF_NAME = pd.DataFrame.from_dict(dict, orient='index')
pd.set_option('display.max_colwidth', None)
df_DF_NAME



,Price,Schoolname,Distance to school,School-grades,Trivsel,Boligtype,Energy-classification,Radon,Kommuneskat,Grundskyldspromillen,...,Ejerudgift,Rooms,Build,Toilets,Postal code,link,Elevator,Altan,Terasse,Toilet
"Hjulmagervej 2,9300",2.298.000 kr.,Sæby Skole,348 m,Ikke oplyst,Ikke oplyst,Villa,Energimærke B,Lav,"26,2%","29,9‰.",...,Ejerudg.: 2.316 kr/md,5 værelser,Opført 1986,3 toiletter,9300,https://www.boligsiden.dk/addresses/0a3f50c9-7f75-32b8-e044-0003ba298018,NaN,NaN,NaN,NaN
"Fuglsang Allé 56,7000",3.545.000 kr.,Erritsø Fællesskole,"2,16 km",Ikke oplyst,Ikke oplyst,Villa,Energimærke A2015,Høj,"25,5%",26‰.,...,Ejerudg.: 2.670 kr/md,5 værelser,Opført 2022,-,7000,https://www.boligsiden.dk/addresses/d3a3a1cf-8764-48b2-8b9a-e7abe150c8f2,NaN,NaN,NaN,NaN
"Spodsbjergvej 239,5900",795.000 kr.,Humble Skole,"11,4 km",6.1,3.7,Villa/Fritidsbolig,Energimærke D,Høj,"26,3%","24,57‰.",...,Ejerudg.: 1.240 kr/md,5 værelser,Opført 1716,1 toilet,5900,https://www.boligsiden.dk/addresses/0a3f50b5-4bf7-32b8-e044-0003ba298018,NaN,NaN,NaN,NaN
"Lemtorpvej 27,7620",4.500.000 kr.,Lemtorpskolen,928 m,7.5,3.9,Villa,Energimærke C,Mellem,"25,46%",28‰.,...,Ejerudg.: 3.686 kr/md,6 værelser,Opført 1898,2 toiletter,7620,https://www.boligsiden.dk/addresses/0a3f50be-760f-32b8-e044-0003ba298018,NaN,NaN,NaN,NaN
"Nordre Digevej 38,2300",6.495.000 kr.,Dyvekeskolen,539 m,5.5,3.8,Rækkehus,Energimærke C,Mellem,"23,7%",34‰.,...,Ejerudg.: 4.912 kr/md,4 værelser,Opført 2005,2 toiletter,2300,https://www.boligsiden.dk/addresses/0a3f509f-cf17-32b8-e044-0003ba298018,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Østerled 46,9370",4.477.000 kr.,Hals Skole,812 m,6.4,3.7,Villa,Energimærke A2010,Meget lav,"25,4%","26,95‰.",...,Ejerudg.: 2.582 kr/md,7 værelser,Opført 1976,2 toiletter,9370,https://www.boligsiden.dk/addresses/0a3f50c7-dbf9-32b8-e044-0003ba298018,NaN,NaN,NaN,NaN
"Urnehovedvej 27,6392",895.000 kr.,Hjordkær Skole,"4,63 km",7,3.7,Villa,Energimærke D,Mellem,"25,6%","18,9‰.",...,Ejerudg.: 1.608 kr/md,4 værelser,Opført 1977,1 toilet,6392,https://www.boligsiden.dk/addresses/0a3f50b7-e90d-32b8-e044-0003ba298018,NaN,NaN,NaN,NaN
"Jordsteds Ager 6,8300",3.495.000 kr.,"Vestskolen, afd. Skovbakken","1,28 km",7.2,3.7,Villa,Energimærke A2010,Mellem,"25,1%","30,32‰.",...,Ejerudg.: 2.606 kr/md,5 værelser,Opført 2016,2 toiletter,8300,https://www.boligsiden.dk/addresses/3cb0a0c6-39dc-0e4a-e044-0003ba298018,NaN,NaN,NaN,NaN
"Rosenvej 15,4800",2.495.000 kr.,SUNDskolen,833 m,6.6,3.7,Villa,Energimærke C,Høj,"25,8%","32,34‰.",...,Ejerudg.: 1.981 kr/md,6 værelser,Opført 1947,2 toiletter,4800,https://www.boligsiden.dk/addresses/0a3f50b0-4371-32b8-e044-0003ba298018,NaN,NaN,NaN,NaN


In [18]:
#Converts to csv-file and save in "Rawdata"
df_DF_NAME.to_csv(f'rawdata/{DF_NAME}.csv', index=True)

